Group 5: LI LINGYU, LIU YICHAO, WU JINGYAN, YANG QINGSHAN, YE FANGDA

In [35]:
# Load env and data
import pandas as pd
import config

chicago_clean = pd.read_csv(config.TRAIN_FILE_CLEAN2)
chicago_clean.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
chicago_clean

,Date,0_7,0_8,0_9,1_3,1_4,1_6,1_7,1_8,1_9,...,7_7,8_0,8_1,8_2,8_3,8_4,9_0,9_1,9_2,9_3
0,2001-01-01 00:00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,3,1,0,0,0,4,1
1,2001-01-01 01:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2001-01-01 02:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2001-01-01 03:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2001-01-01 04:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210692,2025-01-13 20:00:00,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
210693,2025-01-13 21:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
210694,2025-01-13 22:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
210695,2025-01-13 23:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Construct time features

chicago_clean['Date'] = pd.to_datetime(chicago_clean['Date'], format='%Y-%m-%d %H:%M:%S')
chicago_clean['Hour'] = chicago_clean['Date'].dt.hour
chicago_clean['Day'] = chicago_clean['Date'].dt.day
chicago_clean['Month'] = chicago_clean['Date'].dt.month
chicago_clean['DayOfWeek'] = chicago_clean['Date'].dt.dayofweek
chicago_clean['Year'] = chicago_clean['Date'].dt.year
chicago_clean['IsWeekend'] = chicago_clean['DayOfWeek'].isin([5,6]).astype(int)
chicago_clean['TimeOfDay'] = pd.cut(chicago_clean['Hour'], 
                                   bins=[-1,6,12,18,24], 
                                   labels=['Night','Morning','Afternoon','Evening'], 
                                   right=False)
chicago_clean['Season'] = pd.cut(chicago_clean['Month'],
                                bins=[0, 4, 7, 10, 13],
                                labels=['Spring','Summer','Fall', 'Winter'],
                                right=False)
# Drop initial date column
chicago_clean = chicago_clean.drop('Date', axis=1)
chicago_clean.head()

,0_7,0_8,0_9,1_3,1_4,1_6,1_7,1_8,1_9,2_3,...,9_2,9_3,Hour,Day,Month,DayOfWeek,Year,IsWeekend,TimeOfDay,Season
0,0,1,0,0,0,0,0,0,0,0,...,4,1,0,1,1,0,2001,0,Night,Spring
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,2001,0,Night,Spring
2,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,1,0,2001,0,Night,Spring
3,0,0,0,0,0,0,0,0,0,0,...,0,0,3,1,1,0,2001,0,Night,Spring
4,0,0,0,0,0,0,0,0,0,0,...,0,0,4,1,1,0,2001,0,Night,Spring


In [10]:
#construct spatial features
chicago_clean[['grid_x', 'grid_y']] = chicago_clean['grid'].str.split('_', expand=True).astype(int)
chicago_clean['is_border'] = (
    (chicago_clean['grid_x'].isin([0, 10])) | 
    (chicago_clean['grid_y'].isin([0, 10]))
).astype(int)

KeyError: 'grid'

In [ ]:
#construct environmental features

top_locations = chicago['Location Description'].value_counts().head(5).index.tolist()
for loc in top_locations:
    chicago_clean[f'Location_{loc}'] = (chicago['Location Description'] == loc).astype(int)

chicago_clean['Community_Area'] = chicago['Community Area']

chicago_clean['District'] = chicago['District']
chicago_clean['Ward'] = chicago['Ward']

fbi_risk_mapping = {
    '04': 'High',   
    '08': 'Medium', 
    '06': 'Low'     
}
chicago_clean['FBI_Risk'] = chicago['FBI Code'].map(fbi_risk_mapping)
fbi_risk_dummies = pd.get_dummies(chicago_clean['FBI_Risk'], prefix='FBI_Risk')
chicago_clean = pd.concat([chicago_clean, fbi_risk_dummies], axis=1)
chicago_clean = chicago_clean.drop('FBI_Risk', axis=1)


In [ ]:
chicago_clean = pd.get_dummies(chicago_clean, columns=["Season", "TimeOfDay", "DayOfWeek"])
chicago_clean